In [1]:
# Binary Classification with flu Dataset, target outcome:GeneXpert
from tensorflow_addons.optimizers import AdamW
import pandas as pd,tensorflow as tf,keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import StratifiedKFold,cross_val_score,train_test_split
from sklearn.metrics import roc_curve, auc,make_scorer,accuracy_score,precision_score,recall_score,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,LabelEncoder
import os,time,scipy,sherpa,numpy as np
import sherpa.algorithms.bayesian_optimization as bayesian_optimization

from matplotlib import pyplot
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
from termcolor import colored, cprint
from tensorflow.keras import regularizers
print(tf.__version__)

np.set_printoptions(threshold=100000)
pd.set_option('display.width',None)

2.2.0


In [2]:
S_3=pd.read_csv('dataset/Sepsis3_rf_top20_data.csv')
S_1=pd.read_csv('dataset/Siover1_rf_top20_data.csv')
S_7=pd.read_csv('dataset/Sipt7_rf_top20_data.csv')
from utils.dataloader import csv_loader
Seps3,Sio1,Sipt7,data_shape=csv_loader(S_3,S_1,S_7)


Seps3_x=Seps3.iloc[:,1:data_shape[0]]
Seps3_y=Seps3['sofa3m']

Siov1_x=Sio1.iloc[:,1:data_shape[1]]
Siov1_y=Sio1['si1m']

Sipt7_x=Sipt7.iloc[:,1:data_shape[2]]
Sipt7_y=Sipt7['si0.7m']
print(Seps3_x.shape[1])
print(Seps3_y.shape)
print(Seps3_x.shape)

20
(4314,)
(4314, 20)


In [3]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(Siov1_x,Siov1_y,test_size = 0.25, random_state = 1234, stratify =Seps3_y)


In [4]:
import numpy as np
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

In [5]:
parameters = [sherpa.Choice('Deacy',[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]),
              sherpa.Choice('LR',[1e-1,1e-2,1e-3,1e-4]),
              sherpa.Choice('Units',[30,40,50,60,70,80,90,100,100,110,120,130,140]),
              sherpa.Choice('LM',[0,1,2,3]),
#               sherpa.Choice('MM',[0.7,0.8,0.9,0.99,0.999,0.9999]),
              sherpa.Choice('AF',['relu','tanh']),
              sherpa.Choice('Batch',[80,90,100,110,120,130,140,150,160,170,180,190,200]),
              sherpa.Choice('Dropout',[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])]
algorithm = bayesian_optimization.GPyOpt(max_num_trials=1200)
study = sherpa.Study(parameters=parameters,algorithm=algorithm,lower_is_better=False)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://127.0.1.1:8881 if on a cluster or
http://localhost:8881 if running locally.
-------------------------------------------------------


 * Serving Flask app "sherpa.app.app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [6]:
import numpy
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adamax,RMSprop,Adagrad,Adam,Adadelta,Nadam
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed = 1234
numpy.random.seed(seed)
epochs=1
start=time.time()
trials_count=0
for trial in study:
    batch=trial.parameters['Batch']
    decay = trial.parameters['Deacy']
#     mm=trial.parameters['MM']  
    lr=trial.parameters['LR']
    units=trial.parameters['Units']
    layers=trial.parameters['LM']
    dropout=trial.parameters['Dropout']
    AF=trial.parameters['AF']

    model = Sequential()
    model.add(Dense(units, input_dim=Seps3_x.shape[1],kernel_initializer='he_uniform')) 
#                     kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01), 
#                     bias_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(AF))
    model.add(Dropout(dropout))

    for _ in range(int(layers)):
        model.add(Dense(units, kernel_initializer='he_uniform')) 
        model.add(BatchNormalization())
        model.add(Activation(AF))
        model.add(Dropout(dropout))
        
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer=RMSprop(lr=lr,decay=decay)
#     optimizer=AdamW(learning_rate=lr,weight_decay=decay)
#     optimizer=Nadam(lr=lr,schedule_decay=decay)
#     optimizer=Adam(learning_rate=lr,decay=decay)
#     optimizer=Adamax(lr=lr, decay=decay)
#     optimizer = SGD(lr=lr, decay=decay,momentum=mm)
    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    callbacks_list=[EarlyStopping(monitor='val_loss', patience=20, verbose=2, mode='min')]    
    

    history=model.fit(train_x, train_y, epochs=int(300),shuffle=True, batch_size=batch,
                      verbose=0,validation_split=0.1,callbacks=callbacks_list)
    test = model.predict(test_x, verbose=2)
    train = model.predict(train_x, verbose=2)
    auc_test = roc_auc_score(test_y, test)
    auc_train = roc_auc_score(train_y, train)
    study.add_observation(trial=trial, iteration=epochs,
                              objective=auc_test,
                              context={'train_auc':auc_train,
                                       'train_loss':min(history.history['loss']),
                                       'train_acc':max(history.history['accuracy'])})
    if study.should_trial_stop(trial):
        break
    trials_count+=1
    print("\ntrials count: %d"%(trials_count))
    study.finalize(trial)
end=time.time()
print("total time %f sec"%(end-start))

Epoch 00054: early stopping
34/34 - 0s
102/102 - 0s

trials count: 1
Epoch 00025: early stopping
34/34 - 0s
102/102 - 0s

trials count: 2
Epoch 00037: early stopping
34/34 - 0s
102/102 - 0s

trials count: 3
Epoch 00074: early stopping
34/34 - 0s
102/102 - 0s

trials count: 4
34/34 - 0s
102/102 - 0s

trials count: 5
Epoch 00070: early stopping
34/34 - 0s
102/102 - 0s

trials count: 6
Epoch 00117: early stopping
34/34 - 0s
102/102 - 0s

trials count: 7
Epoch 00021: early stopping
34/34 - 0s
102/102 - 0s


INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters



trials count: 8
Epoch 00062: early stopping
34/34 - 0s
102/102 - 0s


INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters



trials count: 9
Epoch 00064: early stopping
34/34 - 0s
102/102 - 0s


...


trials count: 1197
Epoch 00137: early stopping
34/34 - 0s
102/102 - 0s


INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters



trials count: 1198
Epoch 00073: early stopping
34/34 - 0s
102/102 - 0s


INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters



trials count: 1199
Epoch 00077: early stopping
34/34 - 0s
102/102 - 0s

trials count: 1200
total time 59207.712719 sec
